In [1]:
import time
import scipy.io as scio
import numpy as np
from multiprocessing import Pool
import sys, os
from copy import copy
from scipy import stats
from scipy.special import logsumexp
import matplotlib.pyplot as plt
from MDPD import utils, readers, MDPD
import pickle
from ete3 import Tree, faces, TreeStyle

---
## Utilities

In [2]:
def show_img(arrays, fig_width=2):
    "show image(s) in parallel"
    n = len(arrays)
    plt.figure(figsize=(fig_width, fig_width * n))
    for i in range(n):
        plt.subplot(n, 1, i+1)
        plt.imshow(arrays[i].reshape((28,28)))
        plt.colorbar()
    plt.show()

In [3]:
def show_comps(model):
    "show all the components of the model"
    ncomp = model.ncomp
    plt.figure(figsize=(2,2 * ncomp))
    for k in range(ncomp):
        plt.subplot(ncomp, 1, k+1)
        img = np.exp(model.logC[:, 0, k])
        plt.imshow(img.reshape((28, 28)))
        plt.colorbar()
    plt.show()

In [4]:
def show_features(model):
    ""
    img = np.zeros(DIM)
    img[model.features] = 1
    show_img([img])
    

In [5]:
def show_stats(tmp_folder, fig_width=10):
    with open(os.path.join(tmp_folder, 'training_stats.p'), 'rb') as f:
        stats = cPickle.load(f)
    
    plt.figure(figsize=(fig_width, 0.3* fig_width * 2))
    
    plt.subplot(2,1,1)
    plt.plot(stats['log_likelihood'])
    plt.title('log_likelihood')
    
    plt.subplot(2,1,2)
    plt.plot(stats['log_likelihood_overall'])
    plt.title('log_likelihood_overall')
    
    plt.show()

In [6]:
def save_figures_all_models(experiment_folder):
    "save model's visualizations to experiment_folder/images"
    image_dir = os.path.join(experiment_folder, 'images')
    try:
        os.mkdir(image_dir)
    except:
        pass
    
    model = MDPD.Hierachical_MDPD(1)
    model.load(os.path.join(experiment_folder, 'model.p'))
    
    for idx in range(len(model.models)):
        width = model.width
        paren = int((idx - 1) / width)
        kid = idx - paren * width
        
        plt.figure()

        plt.subplot(1,3,1)
        img = model.models[idx].logC[:, 0, :] + model.models[idx].logW[None, :]
        img = np.exp(logsumexp(img,axis=1))
        plt.imshow(img.reshape((28,28)))

        plt.subplot(1,3,2)
        score = utils.Feature_Selection.MI_score(data, sample_log_weights=model._debug[idx], rm_diag=True)
        sigma = score.sum(axis=1) / (DIM-1)
        plt.imshow(sigma.reshape((28, 28)))

        plt.subplot(1,3,3)
        img = np.zeros(sigma.shape)
        cand = np.argsort(sigma)[::-1]
        img[model.models[idx].features] = 1
        plt.imshow(img.reshape((28,28)))

        plt.savefig(os.path.join(image_dir, '{}_{}_{}'.format(idx, paren, kid)), 
                    bbox_inche='tight', transparent=True)
        plt.close()
        

In [40]:
def show_tree(experiment_folder):
    
    model = MDPD.Hierachical_MDPD(1)
    model.load(os.path.join(experiment_folder, 'model.p'))
    
    width, depth = model.width, model.depth
    
    root = Tree()
    
    cache = [(0, root)]
    
    for i in range(depth+1):
        foo = []
        
        for idx, node in cache:
            paren = int((idx-1) / width)
            kid = idx - paren*width
            face = faces.ImgFace(os.path.join(experiment_folder, 'images', '{}_{}_{}.png'.format(idx, paren, kid)))
            node.add_face(face, 0)
            
            if i < depth:
                for k in range(width):
                    foo.append((idx*width + k + 1, node.add_child()))
                    
        cache = foo
    
    root.render(os.path.join(experiment_folder, 'images', 'tree_plot.png'))

---
## Read Data

In [7]:
folder = "/media/vzhao/Data/MNIST"
# folder = "/Users/vincent/Documents/Research/MDPD/MNIST"
mnist = readers.MNIST_Reader(folder, binarized=True)
train, labels = mnist.train, mnist.labels
_, DIM, _ = train.shape

Extracting /media/vzhao/Data/MNIST/train-images-idx3-ubyte.gz
Extracting /media/vzhao/Data/MNIST/train-labels-idx1-ubyte.gz
Extracting /media/vzhao/Data/MNIST/t10k-images-idx3-ubyte.gz
Extracting /media/vzhao/Data/MNIST/t10k-labels-idx1-ubyte.gz


In [8]:
# data per digit
train_uni = [None] * 10
for dig in range(10):
    train_uni[dig] = train[labels==dig,...]
# small sample
train_small = train[:20000,...]
labels_small = labels[:20000]

### Pick a data source

In [9]:
data, labs = train_small, labels_small

---
## Information residue as in raw data

In [ ]:
%%time
score_origin = utils.Feature_Selection.MI_score(data, rm_diag=True)
sigma_origin = score_origin.sum(axis=1)
show_img([sigma_origin])
print 'Information residue in raw data'
print sigma_origin.mean() / (DIM - 1)

#### Reference G score

In [ ]:
percentages = [99,95,90,75,50]
percentiles = [stats.chi2.ppf(x/100.,3) / (2 * data.shape[0]) for x in percentages]
print 'Reference G statistis at {} percentile'.format(percentages)
print percentiles

### Mutual Information Residue if ues the labels as the posterior distribution

In [ ]:
# label to log_post
def label2logpost(label, ncomp):
    nsample = label.shape[0]
    post = np.zeros((nsample, ncomp))
    for i in xrange(nsample):
        post[i, label[i]] = 1
    return np.log(post)
log_post = label2logpost(labs,labs.max()+1)
utils.log_replace_neginf(log_post)

In [ ]:
%%time
score, weighted = MDPD.utils.Feature_Selection.MI_score_conditional(data, log_post, rm_diag=True)
sigma_condition = score.sum(axis=1)
print 'Mutual Information Residue if use the true label as the posterior distribution'
print np.sum(sigma_condition * weighted[np.newaxis, :]) / (DIM * (DIM - 1))

#### [Plot] Mutual Information Residue vs the Residue of the Raw Data

In [ ]:
plt.figure()
idx = np.argsort(sigma_origin)[::-1]
for k in [0]:
    plt.plot(sigma_condition[idx,k]/(DIM-1))
plt.plot(sigma_origin[idx] / (DIM-1), '--')
# plot reference G statistics
for foo in percentiles[:3]:
    plt.plot([0, len(score)], [foo, foo], 'c--')
plt.show()

In [ ]:
plt.figure()
plt.plot(sigma_origin[idx] / (DIM-1), '--')
plt.plot(np.sum(sigma_condition[idx, ...] * weighted[np.newaxis, :], axis=1) / (DIM-1))
# plot reference G statistics
for foo in percentiles[:3]:
    plt.plot([0, len(score)], [foo, foo], 'c--')
plt.show()

#### [Plot] Mutual information residue conditional on a digit vs Raw MIS

In [ ]:
show_img([sigma_origin, sigma_condition[:,1]])

#### Conditional MIS vs Raw MIS

In [ ]:
show_img([sigma_origin, np.sum(sigma_condition * weighted[np.newaxis, :], axis=1)])

#### Conclusion

A naive mixture model is not a good generative model of MNIST data set.

---
## Train a MDPD with the selected features

#### Batch EM

In [ ]:
Ntop = 300
model_batch = MDPD.MDPD_standard()
model_batch.fit(data, ncomp=10, init='random', verbose=False, features=Ntop, epoch=100)

In [ ]:
log_post = model_batch.log_posterior(data)
score, weights = utils.Feature_Selection.MI_score_conditional(data, log_post, rm_diag=True)
sigma_condition = score.sum(axis=1)
print 'Mutual Information Residue of the model with feature selection'
print np.sum(sigma_condition * weights[np.newaxis, :]) / (DIM * (DIM - 1))

#### [plot] mixture component

In [ ]:
show_img(np.exp([model_batch.logC[:,0,i] for i in xrange(10)]), figsize=(50,50))

#### [Plot] Conditional Information Residue vs the Residue of the Raw Data

In [ ]:
plt.figure()
idx = np.argsort(sigma_origin)[::-1]
for k in xrange(3):
    plt.plot(sigma_condition[idx,k]/(DIM-1))
plt.plot(sigma_origin[idx] / (DIM-1), '--')
# plot reference G statistics
for foo in percentiles[:3]:
    plt.plot([0, len(score)], [foo, foo], 'c--')
plt.show()

In [ ]:
plt.figure()
plt.plot(sigma_origin[idx] / (DIM-1), '--')
plt.plot(np.sum(sigma_condition[idx, ...] * weighted[np.newaxis, :], axis=1) / (DIM-1))
# plot reference G statistics
for foo in percentiles[:3]:
    plt.plot([0, len(score)], [foo, foo], 'c--')
plt.show()

#### [Plot] Mutual information residue conditional on a digit vs Raw MIS

In [ ]:
show_img([sigma_origin, sigma_condition[:,3]])

In [ ]:
show_img([sigma_origin, np.sum(sigma_condition * weighted[np.newaxis, :], axis=1)])

---
## Hierachical MDPD

In [42]:
experiment_folder = '/home/vzhao/Documents/Projects/MDPD/results/MNIST_hmdpd_depth_6'

In [43]:
model = MDPD.Hierachical_MDPD(1)
model.load(os.path.join(experiment_folder, 'model.p'))

UnicodeDecodeError: 'ascii' codec can't decode byte 0xc7 in position 8: ordinal not in range(128)

#### Show the component of leaf models

In [ ]:
for k in range(20):
    show_comps(model.models[-k-1])

#### Analysis of one leaf model

**components**

In [ ]:
idx = 61
width = model.width
paren = int((idx - 1) / width)
kid = idx - paren * width

### mean
img = model.models[idx].logC[:, 0, :] + model.models[idx].logW[None, :]
img = np.exp(logsumexp(img,axis=1))
show_img([img])

### components
show_comps(model.models[idx])

### feature sets
# inf_path = model.inference_path(data)
# sample_log_weight = inf_path[idx][:,0]
# sample_log_weight = sample_log_weight - logsumexp(sample_log_weight)
# score = utils.Feature_Selection.MI_score(data, sample_log_weights=sample_log_weight, rm_diag=True)
# sigma = score.sum(axis=1) / (DIM-1)
# show_img([sigma], fig_width=2)

score = utils.Feature_Selection.MI_score(data, sample_log_weights=model._debug[idx], rm_diag=True)
sigma = score.sum(axis=1) / (DIM-1)
show_img([sigma], fig_width=2)
img = np.zeros(sigma.shape)
cand = np.argsort(sigma)[::-1]
img[model.models[idx].features] = 1
show_img([img])

In [ ]:
save_figures_all_models(experiment_folder)

**Tree plot**

In [13]:
model.depth

5

In [41]:
root = show_tree(experiment_folder)
    


---
## SandBox

In [ ]:
score = utils.Feature_Selection.MI_score(data, rm_diag=True)
sigma = score.sum(axis=1) / (DIM-1)
cand = np.argsort(sigma)[::-1]

In [ ]:
topN = 50

In [ ]:
%%time
model_online = MDPD.MDPD_online()
model_online.fit(data, 2, features=cand[:topN], init='random', epoch=20, batch=500, verbose=False)

In [ ]:
print len(model_online.features)
show_comps(model_online)
show_features(model_online)

In [ ]:
tmp_folder = '/home/vzhao/Documents/Projects/MDPD/tmpyfxdvd'
show_stats(tmp_folder)

In [ ]:
%%time
model_standard = MDPD.MDPD_standard()
model_standard.fit(data, 2, features=cand[:topN], init='random', epoch=20, verbose=False)

In [ ]:
# with open(os.path.join(tmp_folder, 'training_stats.p'), 'rb') as f:
#     stats = cPickle.load(f)

In [ ]:
tmp_folder = '/home/vzhao/Documents/Projects/MDPD/tmplbNHtY'
show_stats(tmp_folder)

In [ ]:
print len(model_standard.features)
show_comps(model_standard)
show_features(model_standard)

In [ ]:
tmp_folder = ''

In [ ]:
model_standard = MDPD.MDPD_standard()
model_standard.fit(data, 2, features=cand[:topN], init='random', epoch=100, verbose=True)

In [ ]:
print len(model_standard.features)
show_comps(model_standard)
show_features(model_standard)